# Lyrics genre single label classification (HAN with glove embeddings)

This notebook:
* Uses the dataset generated by **helgi-05-scraped-lyrics-v2-preprocessing.ipynb**
* Runs the HAN model implemented here https://github.com/tqtg/hierarchical-attention-networks using glove embeddings
* Tensorflow version 1.13.1 was used in this notebook (closest version available to when the HAN was implemented)

In [1]:
import random
import ast
from datetime import datetime

import numpy as np
import pandas as pd

import sklearn.utils

import tensorflow as tf
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow_hub as hub
from tensorflow.python.client import device_lib

import nltk.tokenize

random_state = 1111
np.random.seed(random_state)
random.seed(random_state)

import sys
sys.path.append("../")

import pandas as pd
import nltk
import itertools
import pickle

from han.data_reader import DataReader
from han.model import Model
from han.utils import read_vocab, count_parameters, load_glove


E:\Repos\comp550-final-project\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Repos\comp550-final-project\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Repos\comp550-final-project\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Repos\comp550-final-project\venv\lib\site-packages\tensorflow\python\f

## Load and prepare data

In [2]:
df = pd.read_csv('E:\\Repos\\comp550-final-project\\data\\scraped-lyrics-v2-preprocessed.csv')
df

,artist,song,lyrics,genres,category
0,12 Stones,3 Leaf Loser,"Biting the hand that feeds you, lying to the v...","['Hard Rock', 'Rock']",Hard Rock
1,12 Stones,Adrenaline,My heart is beating faster can't control these...,"['Hard Rock', 'Rock']",Hard Rock
2,12 Stones,Anthem For The Underdog,You say you know just who I am\nBut you can't ...,"['Hard Rock', 'Rock']",Hard Rock
3,12 Stones,Anthem For The Underdog (Picture Perfect Sessi...,You say you know just who I am\nBut you can't ...,"['Hard Rock', 'Rock']",Hard Rock
4,12 Stones,Arms Of A Stranger,"I came home early to see you,\nCouldn't wait t...","['Hard Rock', 'Rock']",Hard Rock
...,...,...,...,...,...
58714,Ängie,Spun,"Avoid eye contact, you want it, I know it's a ...",['Indie'],Indie
58715,Ängie,Talk to Me Nice,Imma fuck you up\nImma fuck you up\nImma fuck ...,['Indie'],Indie
58716,Ängie,Two Together,I don't even think that you're my type\nYou're...,['Indie'],Indie
58717,Ängie,Venus In Furs,"Shiny, shiny, shiny boots of leather\nWhiplash...",['Indie'],Indie


In [3]:
# genres are a string, let's convert to a list
df.genres = df.genres.apply(lambda x: ast.literal_eval(x))

In [4]:
data_ratio = 1 # How much of the data to use
df.index.to_numpy()
splits = [0.7, 0.85]

indices = sklearn.utils.shuffle(df.index.to_numpy(), random_state=1234)
n_data_used = int(data_ratio*len(indices))

n_categories = df.category.nunique()
category_to_int = {category: i+1 for i, category in enumerate(df.category.unique())}

In [ ]:
all_data = []
for i in indices[ : n_data_used]:
    song = df.iloc[i]
    tokenized_sentences = [nltk.tokenize.word_tokenize(sentence) for sentence in song.lyrics.lower().split('\n')]
    all_data.append((tokenized_sentences, category_to_int[song.category]))

all_data = np.array(all_data, dtype=object)
n = len(all_data)
data = {
    'X_train': all_data[:int(splits[0]*n), 0],
    'y_train': all_data[:int(splits[0]*n), 1],
    'X_valid': all_data[int(splits[0]*n):int(splits[1]*n), 0],
    'y_valid': all_data[int(splits[0]*n): int(splits[1]*n), 1],
    'X_test': all_data[int(splits[1]*n):, 0],
    'y_test': all_data[int(splits[1]*n):, 1]
}

print(f'Ratio of data used: {data_ratio}*{len(indices)} = {n_data_used}')

## Tensorflow installation verification

In [5]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

## HAN data preparation

In [6]:
han_directory = 'E:/Repos/comp550-final-project/data/han'

In [ ]:
# Hyper parameters
WORD_CUT_OFF = 5

def build_vocab(docs, save_path):
    print('Building vocab ...')

    # Count the word frequencies
    word_freq = nltk.FreqDist(itertools.chain(*[sentence for doc in data['X_train'] for sentence in doc]))
    print("%d unique words found" % len(word_freq.items()))

    # Cut-off
    retained_words = [w for (w, f) in word_freq.items() if f > WORD_CUT_OFF]
    print("%d words retained" % len(retained_words))

    # Get the most common words and build index_to_word and word_to_index vectors
    # Word index starts from 2, 1 is reserved for UNK, 0 is reserved for padding
    word_to_index = {'PAD': 0, 'UNK': 1}
    for i, w in enumerate(retained_words):
        word_to_index[w] = i + 2
        index_to_word = {i: w for (w, i) in word_to_index.items()}

    print("Vocabulary size = %d" % len(word_to_index))

    with open(f'{save_path}/w2i.pkl', 'wb+') as f:
        pickle.dump(word_to_index, f)

    with open(f'{save_path}/i2w.pkl', 'wb+') as f:
        pickle.dump(index_to_word, f)

    return word_to_index

def process_and_save(word_to_index, X, y, out_file):
    assert len(X) == len(y)
    mapped_data = []
    for label, doc in zip(y, X):
        mapped_doc = [[word_to_index.get(word, 1) for word in sentence] for sentence in doc]
        mapped_data.append((label, mapped_doc))

    with open(out_file, 'wb') as f:
        pickle.dump(mapped_data, f)

word_to_index = build_vocab(data['X_train'], f'{han_directory}')
process_and_save(word_to_index, data['X_train'], data['y_train'], f'{han_directory}/train.pkl')
process_and_save(word_to_index, data['X_valid'], data['y_valid'], f'{han_directory}/valid.pkl')
process_and_save(word_to_index, data['X_test'], data['y_test'], f'{han_directory}/test.pkl')


## HAN model training

In [7]:
def loss_fn(labels, logits):
    onehot_labels = tf.one_hot(labels, depth=FLAGS.num_classes)
    cross_entropy_loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels,
                                                         logits=logits)
    tf.summary.scalar('loss', cross_entropy_loss)
    return cross_entropy_loss


def train_fn(loss):
    trained_vars = tf.trainable_variables()
    count_parameters(trained_vars, print_summary=False)

    # Gradient clipping
    gradients = tf.gradients(loss, trained_vars)

    clipped_grads, global_norm = tf.clip_by_global_norm(
        gradients, FLAGS.max_grad_norm)
    tf.summary.scalar('global_grad_norm', global_norm)

    # Define optimizer
    global_step = tf.train.get_or_create_global_step()
    optimizer = tf.train.RMSPropOptimizer(FLAGS.learning_rate)
    train_op = optimizer.apply_gradients(zip(clipped_grads, trained_vars),
                                         name='train_op',
                                         global_step=global_step)
    return train_op, global_step


def eval_fn(labels, logits):
    predictions = tf.argmax(logits, axis=-1)
    correct_preds = tf.equal(predictions, tf.cast(labels, tf.int64))
    batch_acc = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
    tf.summary.scalar('accuracy', batch_acc)

    total_acc, acc_update = tf.metrics.accuracy(
        labels, predictions, name='metrics/acc')
    metrics_vars = tf.get_collection(
        tf.GraphKeys.LOCAL_VARIABLES, scope="metrics")
    metrics_init = tf.variables_initializer(var_list=metrics_vars)

    return batch_acc, total_acc, acc_update, metrics_init


def train_model():
    config = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement)
    with tf.Session(config=config) as sess:
        tf.set_random_seed(1234)
        model = Model(cell_dim=FLAGS.cell_dim,
                      att_dim=FLAGS.att_dim,
                      vocab_size=len(vocab),
                      emb_size=FLAGS.emb_size,
                      num_classes=FLAGS.num_classes,
                      dropout_rate=FLAGS.dropout_rate,
                      pretrained_embs=glove_embs)

        loss = loss_fn(model.labels, model.logits)
        train_op, global_step = train_fn(loss)
        batch_acc, total_acc, acc_update, metrics_init = eval_fn(
            model.labels, model.logits)
        sess.run(tf.global_variables_initializer())

        epoch = 0
        train_test_prop = len(data_reader.train_data) / \
            len(data_reader.test_data)
        test_batch_size = int(FLAGS.batch_size / train_test_prop)

        while epoch < FLAGS.num_epochs:
            epoch += 1
            sess.run(metrics_init)
            for batch_docs, batch_labels in data_reader.read_train_set(FLAGS.batch_size, shuffle=True):
                _step, _, _loss, _acc, _ = sess.run([global_step, train_op, loss, batch_acc, acc_update],
                                                    feed_dict=model.get_feed_dict(batch_docs, batch_labels, training=True))

            sess.run(metrics_init)
            for batch_docs, batch_labels in data_reader.read_valid_set(test_batch_size):
                _loss, _acc, _ = sess.run(
                    [loss, batch_acc, acc_update], feed_dict=model.get_feed_dict(batch_docs, batch_labels))
            print('Validation accuracy = {:.3f}'.format(
                sess.run(total_acc)), flush=True)

            sess.run(metrics_init)
            for batch_docs, batch_labels in data_reader.read_test_set(test_batch_size):
                _loss, _acc, _ = sess.run(
                    [loss, batch_acc, acc_update], feed_dict=model.get_feed_dict(batch_docs, batch_labels))
                test_acc = sess.run(total_acc) * 100
            test_acc = sess.run(total_acc)
            print('Testing accuracy = {:.3f}'.format(test_acc), flush=True)


In [8]:
class Parameters:
    def __init__(self, cell_dim=50, att_dim=100, emb_size=200, max_sentences=30, max_sentence_words=50,
                 num_classes=n_categories, num_epochs=20, batch_size=64, learning_rate=0.0005,
                 max_grad_norm=5.0, dropout_rate=0.5, allow_soft_placement=True):
        """
        cell_dim: Hidden dimensions of GRU cells
        att_dim: Dimensionality of attention spaces
        emb_size: Dimensionality of word embedding
        max_sentences: Maximum sentences per document
        max_sentence_words: Maximum words per sentence
        num_classes: Number of classes
        num_epochs: Number of training epochs
        batch_size: Batch size
        learning_rate: Learning rate
        max_grad_norm: Maximum value of the global norm of the gradients for clipping
        dropout_rate: Probability of dropping neurons
        allow_soft_placement: Allow device soft device placement
        """
        self.cell_dim = cell_dim
        self.att_dim = att_dim
        self.emb_size = emb_size
        self.max_sentences = max_sentences
        self.max_sentence_words = max_sentence_words
        self.num_classes = num_classes
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.max_grad_norm = max_grad_norm
        self.dropout_rate = dropout_rate
        self.allow_soft_placement = allow_soft_placement
    
    def __str__(self):
        return f'cell_dim={self.cell_dim}, att_dim={self.att_dim}, emb_size={self.emb_size}, max_sentences={self.max_sentences}, '\
        + f'max_sentence_words={self.max_sentence_words}, num_classes={n_categories}, num_epochs={self.num_epochs}, '\
        + f'batch_size={self.batch_size}, learning_rate={self.learning_rate}, max_grad_norm={self.max_grad_norm}, '\
        + f'dropout_rate={self.dropout_rate}, allow_soft_placement={self.allow_soft_placement}'


In [9]:
parameters = [
    Parameters(cell_dim=50, att_dim=100, emb_size=50, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=50, att_dim=100, emb_size=100, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=50, att_dim=100, emb_size=200, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=50, att_dim=100, emb_size=300, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=75, att_dim=125, emb_size=50, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=75, att_dim=125, emb_size=100, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=75, att_dim=125, emb_size=200, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=75, att_dim=125, emb_size=300, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=100, att_dim=150, emb_size=50, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=100, att_dim=150, emb_size=100, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=100, att_dim=150, emb_size=200, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
    Parameters(cell_dim=100, att_dim=150, emb_size=300, max_sentences=50, max_sentence_words=50, num_epochs=15, batch_size=32, learning_rate=0.0005),
]

for i in range(len(parameters)):
    print('=' * 100)
    print(f'Model {i+1}/{len(parameters)}')
    FLAGS = parameters[i]
    print(FLAGS)
    print('=' * 100)
    
    vocab = read_vocab(f'{han_directory}/w2i.pkl')
    glove_embs = load_glove(f'{han_directory}/glove/glove.6B.{FLAGS.emb_size}d.txt', FLAGS.emb_size, vocab)
    data_reader = DataReader(train_file=f'{han_directory}/train.pkl',
                             valid_file=f'{han_directory}/valid.pkl',
                             test_file=f'{han_directory}/test.pkl',
                             max_word_length=FLAGS.max_sentence_words,
                             max_sent_length=FLAGS.max_sentences,
                             num_classes=FLAGS.num_classes)
    
    with tf.device('/gpu:0'):
        tf.reset_default_graph()
        train_model()


Model 1/12
cell_dim=50, att_dim=100, emb_size=50, max_sentences=50, max_sentence_words=50, num_classes=10, num_epochs=15, batch_size=32, learning_rate=0.0005, max_grad_norm=5.0, dropout_rate=0.5, allow_soft_placement=True
Loading vocabulary ...
Vocabulary size = 25677
Loading Glove pre-trained word embeddings ...
Total 400000 word vectors in E:/Repos/comp550-final-project/data/han/glove/glove.6B.50d.txt
Number of OOV words = 3495
Reading data from E:/Repos/comp550-final-project/data/han/train.pkl
Reading data from E:/Repos/comp550-final-project/data/han/valid.pkl
Reading data from E:/Repos/comp550-final-project/data/han/test.pkl
Total trainable parameters: 1381060


Validating: 100%|██████████| 1468/1468 [00:38<00:00, 38.41it/s]                                                                                                                                              

Validation accuracy = 0.360



Testing: 100%|██████████| 1468/1468 [00:38<00:00, 38.36it/s]                                                                                                                                                 

Testing accuracy = 0.361


...

Testing accuracy = 0.467



Validating: 100%|██████████| 1468/1468 [00:38<00:00, 37.77it/s]                                                                                                                                              

Validation accuracy = 0.472



Testing: 100%|██████████| 1468/1468 [00:39<00:00, 37.24it/s]                                                                                                                                                 

Testing accuracy = 0.470
